In [1]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_processed_data, load_split_processed_data
from utils.model import predict, load_model, predict_booster, predict_booster_model
from steps.load_data import LoadData

In [45]:
data_loader = LoadData()

# data_loader.df_train_fe.abon_id
df_train_bnum = data_loader.df_train_bnum.copy()

df_train_bnum["target"] = 0

abon_target = data_loader.df_train_fe[["target", "abon_id"]]

df_train_bnum["target"] = df_train_bnum["abon_id"].map(abon_target.set_index("abon_id")["target"])

# Percentage of boolean columns
df_train_bnum["target"].value_counts(normalize=True)

df_train_bnum = df_train_bnum.copy()

In [71]:
# # Fetch phone number information

# import requests
# import json


# short_numbers_dict = {}

# short_numbers = [
#     "111",
#     "1020",
#     "275",
#     "3700",
#     "277",
#     "5010",
#     "3135",
#     "273",
#     "3133",
#     "112",
#     "7777",
#     "1545",
#     "4224",
#     "1525",
#     "30094",
#     "1648",
#     "2233",
#     "555",
#     "1551",
#     "30043",
#     "2250",
#     "1050",
#     "225",
#     "1535",
#     "1648",
#     "5000",
#     "4666",
# ]


# def fetch_phone_information(short_number: str):
#     try:
#         base_url = "https://www.vodafone.ua/api/short_numbers/?short_number={variable}&Voice=on&SMS=on"

#         url = base_url.format(variable=short_number)

#         response = requests.get(url)

#         payload = json.loads(response.content.decode("utf-8"))

#         return payload["data"][0]["title"]
#     except:
#         return "Not found"


# for number in short_numbers:
#     short_numbers_dict[number] = fetch_phone_information(number)

In [93]:
short_numbers_dict

{'111': 'Служба підтримки VODAFONE Україна',
 '1020': 'Автоматичні налаштування Мобільного Інтернету ',
 '275': 'NPS-опитування компанії Vodafone',
 '3700': 'М-Банкінг',
 '277': 'NSAT- опитування компанії Vodafone, про задоволеність обслуговуванням в компанії',
 '5010': 'Гроші на замовлення',
 '3135': 'Служба таксі',
 '273': 'NPS-опитування компанії Vodafone',
 '3133': 'Служба таксі',
 '112': 'Рятувальна служба',
 '7777': 'Міграція в інший тариф Vodafone',
 '1545': 'Державний контактний центр',
 '4224': 'Соціальний проект Ю-Репорт @UReportUkraine',
 '1525': 'Not found',
 '30094': 'Vodafone TV. Пакет PRO',
 '1648': 'ДЕРЖАВНА УСТАНОВА "УРЯДОВИЙ КОНТАКТНИЙ ЦЕНТР"',
 '2233': 'Служба таксі',
 '555': 'Not found',
 '1551': 'Контактний Центр міста Києва',
 '30043': 'Not found',
 '2250': 'Інтерактивний проект ТОВ «Телесвіт» (volia.com)',
 '1050': 'Not found',
 '225': '"Топливо в наявності"',
 '1535': 'Інформаційни портал Харківської міської ради',
 '5000': 'Not found',
 '4666': 'Not found'}

In [88]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import time


# def get_phonebook_header(phone_number):
#     url = "https://www.telefonnyjdovidnyk.com.ua/"

#     # Set up headless Chrome
#     chrome_options = Options()
#     chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--no-sandbox")

#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

#     # Open the URL
#     driver.get(url)

#     # Find the search input, enter the phone number, and submit the form
#     search_input = driver.find_element("name", "search")
#     search_input.send_keys(phone_number)
#     search_input.submit()

#     # Wait for the page to load (you may need to adjust the sleep time)
#     time.sleep(3)

#     # Get the page source and parse it with BeautifulSoup
#     soup = BeautifulSoup(driver.page_source, "html.parser")

#     # Look for the phonebook-header class
#     phonebook_header = soup.find(class_="phonebook-header")

#     # If not found, look for the first h3 tag
#     if not phonebook_header:
#         phonebook_header = soup.find("h3")

#     driver.quit()

#     if phonebook_header:
#         header_text = phonebook_header.get_text(strip=True)
#         return {"header": header_text}
#     else:
#         return {"error": "phonebook-header or h3 not found"}

In [86]:
some_numbers_dict = {}

In [89]:
# for number in some_numbers:
#     print("Fetching", number)

#     if some_numbers_dict.get(number) and some_numbers_dict.get(number) != "phonebook-header or h3 not found":
#         print("Already fetched", some_numbers_dict[number])
#         continue

#     some_numbers_dict[number] = get_phonebook_header(number)
#     print("Fetched", some_numbers_dict[number])

Fetching 380800210800
Already fetched {'header': 'Аферисти Київ'}
Fetching 380800500500
Already fetched {'header': 'РАЙФФАЙЗЕН БАНК АВАЛЬ, ПАТ, ДНІПРОПЕТРОВСЬКАОБЛАС'}
Fetching 380800305555
Already fetched {'header': 'Телефонне шахрайство: у МВС розповіли, хто і як виманює гроші в українців'}
Fetching 380800500850
Already fetched {'header': 'КРЕДОБАНК,АТ'}
Fetching 380800307010
Fetched {'header': 'БАНК ВОСТОК,ПАТ'}
Fetching 380800502050
Fetched {'header': 'SenseBank'}
Fetching 380800504450
Fetched {'header': 'ЕКСПОРТНО-ІМПОРТНИЙ БАНК УКРАЇНИ, ПАТ, ЧЕРНІВЕЦЬКАФІЛІЯ'}
Fetching 380800502030
Fetched {'header': 'ІДЕЯ БАНК,АТ'}
Fetching 380800507700
Fetched {'header': 'БАНК КРЕДИТ ДНІПРО,АТ'}
Fetching 380800504400
Fetched {'header': 'МЕГАБАНК,АТ'}
Fetching 380800309000
Fetched {'header': 'УКРГАЗБАНК, АКЦІОНЕРНИЙБАНК'}
Fetching 380800307030
Fetched {'header': 'ПІВДЕННИЙ, ПУБЛІЧНЕ АТ АКЦІОНЕРНИЙБАНК'}
Fetching 380800500850
Already fetched {'header': 'КРЕДОБАНК,АТ'}
Fetching 380442474040
Fetche

In [90]:
some_numbers_dict = {
    "380800210800": {"header": "Аферисти Київ"},
    "380800500500": {"header": "РАЙФФАЙЗЕН БАНК АВАЛЬ, ПАТ, ДНІПРОПЕТРОВСЬКАОБЛАС"},
    "380800305555": {"header": "Телефонне шахрайство: у МВС розповіли, хто і як виманює гроші в українців"},
    "380800500850": {"header": "КРЕДОБАНК,АТ"},
    "380800307010": {"header": "БАНК ВОСТОК,ПАТ"},
    "380800502050": {"header": "SenseBank"},
    "380800504450": {"header": "ЕКСПОРТНО-ІМПОРТНИЙ БАНК УКРАЇНИ, ПАТ, ЧЕРНІВЕЦЬКАФІЛІЯ"},
    "380800502030": {"header": "ІДЕЯ БАНК,АТ"},
    "380800507700": {"header": "БАНК КРЕДИТ ДНІПРО,АТ"},
    "380800504400": {"header": "МЕГАБАНК,АТ"},
    "380800309000": {"header": "УКРГАЗБАНК, АКЦІОНЕРНИЙБАНК"},
    "380800307030": {"header": "ПІВДЕННИЙ, ПУБЛІЧНЕ АТ АКЦІОНЕРНИЙБАНК"},
    "380442474040": {"header": "КИЇВТЕПЛОЕНЕРГО,КП"},
    "380442907290": {"header": "ПЕРШИЙ УКРАЇНСЬКИЙ МІЖНАРОДНИЙБАНК"},
    "380443630133": {"header": "ОЩАДБАНК, УКРАЇНСЬКИЙ КОМЕРЦІЙНИЙБАНК"},
    "380442020202": {
        "header": "У Черкасах телефонні шахраї заробляють десятки тисяч гривень на довірливості містян"
    },
    "380443519998": {"header": "Інтерес до цього номера за останній місяць"},
    "380443519999": {"header": "ЮНІКАСТ ІНВЕСТ,ТОВ"},
    "380442050020": {"header": "СІНЕВО УКРАЇНА,ТОВ"},
    "380442329925": {"header": "ЗАКАЗ. ЮА,ТОВ"},
    "380442021588": {"header": "КИЇВ ЕНЕРГО,АТ"},
    "380442224222": {"header": "СПОРТ ЛАЙФ УКРАЇНА,ТОВ"},
    "380442470786": {"header": "STIL EVROTOV"},
    "380444590740": {"header": "КИЇВОБЛЕНЕРГО"},
    "380444908888": {"header": "РАЙФФАЙЗЕН БАНК АВАЛЬ,АТ"},
    "380444950405": {"header": "Як не стати жертвою телефонних шахраїв"},
    "380444944320": {"header": "Інтерес до цього номера за останній місяць"},
    "380444900500": {"header": "ОТП БАНК,АТ"},
    "380445038080": {"header": "Це виртуальний оператор"},
    "380445276363": {"header": "Як не стати жертвою телефонних шахраїв. Пояснюють експерти"},
    "380445370222": {"header": "Телефонне платіжне шахрайство –#ШахрайГудбай"},
    "380445001717": {
        "header": "У Черкасах телефонні шахраї заробляють десятки тисяч гривень на довірливості містян"
    },
    "380445000303": {"header": "ХАРДТЕХСЕРВІС,ТОВ"},
    "380442901988": {"header": "Шахрайство в мережі та телефонне шахрайство"},
    "380442221111": {"header": "Як діяти українцям, коли телефонують шахраї"},
    "380442204404": {
        "header": "Обережно: телефонне шахрайство! Чернігівців «розводять» на гроші під час пошуку роботи"
    },
    "380442220333": {
        "header": "КІБЕРПОЛІЦІЯ ЗАСТЕРІГАЄ ВІД ТЕЛЕФОННИХ ШАХРАЇВ, ЯКІ ВИДАЮТЬ СЕБЕ ЗА СПІВРОБІТНИКІВ БАНКУ"
    },
    "380442009010": {"header": "Мошенники “разводят” предпринимателей"},
    "380487059090": {"header": "Інтерес до цього номера за останній місяць"},
    "380487300030": {"header": "ODREX, МЕДИЧНИЙДІМ"},
    "380487055555": {"header": "Телефонне шахрайство: у МВС розповіли, хто і як виманює гроші в українців"},
    "380567934826": {"header": "STANTSIYA DNIPROPETROVSKFILIA"},
    "380567878104": {"header": "На Рівненщині 1,5 року обмеження волі за телефонне шахрайство отримав волинянин"},
    "380567900999": {"header": "Інтерес до цього номера за останній місяць"},
    "380577075757": {"header": "ХАРКІВВОДОКАНАЛ,КП"},
    "380577209720": {"header": "МАКСНЕТ"},
    "380577665104": {"header": "ХАРКІВМІСЬКГАЗ,ПАТ"},
    "380666163133": {"header": "На Рівненщині за зухвале телефонне шахрайство піде під суд житель Миколаївщини"},
    "380670107104": {"header": "ХАРКІВМІСЬКГАЗ,ПАТ"},
    "380674660466": {"header": "Київстар"},
    "380677880884": {"header": "Що треба знати про мобільне шахрайство"},
    "380931710000": {"header": "ЦЕНТР ЦИФРАЛ СЕРВІС ЗАПОРІЖЖЯ,ТОВ"},
    "380931771611": {"header": "ЛАЗЕРХАУЗ,ТОВ"},
    "380952331515": {"header": "ЗАПОРІЗЬКІ ЦИФРОВІ КОМУНІКАЦІЇ,ТОВ"},
    "380957321212": {
        "header": "Які види шахрайства розповсюджені на Житомирщині і як не стати жертвою шахраїв, розповіли у…"
    },
    "380958788181": {
        "header": "КІБЕРПОЛІЦІЯ ЗАСТЕРІГАЄ ВІД ТЕЛЕФОННИХ ШАХРАЇВ, ЯКІ ВИДАЮТЬ СЕБЕ ЗА СПІВРОБІТНИКІВ БАНКУ"
    },
    "380962907290": {"header": "ПЕРШИЙ УКРАЇНСЬКИЙ МІЖНАРОДНИЙБАНК"},
    "380984500609": {"header": "НОВА ПОШТА,КОМПАНІЯ"},
    "380995055757": {
        "header": "НАЦІОНАЛЬНИЙ БАНК УКРАЇНИ ЗАПУСТИВ ІНФОРМАЦІЙНУ КАМПАНІЮ З ПРОТИДІЇ ШАХРАЙСТВУ #ШАХРАЙГУДБАЙ"
    },
    "380997344444": {"header": "ФРЕГАТ ІСП,ТОВ"},
}

len(some_numbers_dict)

{'380800210800': {'header': 'Аферисти Київ'},
 '380800500500': {'header': 'РАЙФФАЙЗЕН БАНК АВАЛЬ, ПАТ, ДНІПРОПЕТРОВСЬКАОБЛАС'},
 '380800305555': {'header': 'Телефонне шахрайство: у МВС розповіли, хто і як виманює гроші в українців'},
 '380800500850': {'header': 'КРЕДОБАНК,АТ'},
 '380800307010': {'header': 'БАНК ВОСТОК,ПАТ'},
 '380800502050': {'header': 'SenseBank'},
 '380800504450': {'header': 'ЕКСПОРТНО-ІМПОРТНИЙ БАНК УКРАЇНИ, ПАТ, ЧЕРНІВЕЦЬКАФІЛІЯ'},
 '380800502030': {'header': 'ІДЕЯ БАНК,АТ'},
 '380800507700': {'header': 'БАНК КРЕДИТ ДНІПРО,АТ'},
 '380800504400': {'header': 'МЕГАБАНК,АТ'},
 '380800309000': {'header': 'УКРГАЗБАНК, АКЦІОНЕРНИЙБАНК'},
 '380800307030': {'header': 'ПІВДЕННИЙ, ПУБЛІЧНЕ АТ АКЦІОНЕРНИЙБАНК'},
 '380442474040': {'header': 'КИЇВТЕПЛОЕНЕРГО,КП'},
 '380442907290': {'header': 'ПЕРШИЙ УКРАЇНСЬКИЙ МІЖНАРОДНИЙБАНК'},
 '380443630133': {'header': 'ОЩАДБАНК, УКРАЇНСЬКИЙ КОМЕРЦІЙНИЙБАНК'},
 '380442020202': {'header': 'У Черкасах телефонні шахраї заробляють десятки тисяч

In [224]:
# Перевірити чи частіше від нас йдуть ті кому дзвонять мошенніки


# df_train_bnum.bnum.value_counts().sort_values(ascending=False).index.to_list()

bank_companies = [
    "privatbank",
    "oschadbank",
    "raiffeisen",
    "monobank",
    "mono",
    "alfabank",
    "pumb",
    "otp bank",
    "ukrgasbank",
    "a-bank",
    "tascombank",
    "ukreximbank",
    "bankvostok",
    "pivdenny",
    "globusbank",
    "ukreximban",
    "poltavaban",
    "bank bls",
    "accordbank",
    "creditdnepr",
    "megabank",
    "concordbank",
    "radabank",
    "procredit",
    "industrial",
    "pravex ban",
    "rws bank",
    "bankago",
    "credobank",
    "forwardban",
    "sportbank",
    "idea bank",
    "mtb bank",
    "piraeusbank",
    "bankforward",
    "alf-ua.com",
    "kredobank",
    "sichbank",
    "izibank",
    "pumb onlin",
    "ideabank",
]

work_companies = [
    "rabota.ua",
    "work.ua",
    "jooble",
    "linkedin",
    "itstep",
    "skillup",
    "rabota",
    "start-work",
]

grocery_companies = [
    "varus",
    "silpo",
    "fozzy",
    "myasomarke",
    "metro",
    "tavria_v",
    "eko market",
    "фрукты",
]

post_companies = [
    "380984500609",
    "novaposhta",
]

taxy_companies = [
    "taxi-838",
    "uklon",
    "bolt",
    "uber",
    "shark taxi",
    "taxi 3040",
    "opti-579",
    "ontaxi",
    "taxi 2288",
    "taxi vezi",
    "taxi 323",
    "taxi 571",
    "taxi-808",
    "eco taxi",
    "taxi-280",
    "taximer",
    "taxi 309",
    "taxi 777",
    "taxi-653",
    "taxi 959",
    "2233",
    "3135",
    "3133",
]

credits_companies = [
    "credit_plus",
    "credit7.ua",
    "mycredit",
    "ze.kredit",
    "moneyveo",
    "mycredit.ua",
    "creditdnepr",
    "e-groshi",
    "credit7",
    "clickcredit",
    "soscredit",
    "crediton",
    "creditexpr",
    "creditdnep",
    "credit_plus",
    "creditplu",
    "creditdebt",
    "zecredit",
    "kredit",
    "alexcredit",
    "shgroshi",
    "creditpod",
    "credit7.ua",
    "money4you",
    "dinero",
    "kreditytut",
    "creditpod-0",
    "clickcredi",
    "kredit4u",
    "e-wings",
    "creditexpr",
    "minizaem",
    "money24",
    "moneyboom",
    "moneyextra",
    "moneylove",
    "zss",
    "creditplus",
    "k-kapital",
    "egroshicom",
    "skarbcomua",
    "credit_plu",
    "miloan.ua",
    "mycredit-u",
    "mycredit-ua",
    "mycredit.u",
    "ua-mycredi",
    "ua-mycredit",
    "mycreditua",
    "ccloan",
    "loanyua",
    "ze.kred1t",
    "selficredi",
    "mistercash",
    "selficredit",
    "sos credit",
    "zecredi",
    "dengi dozp",
    "otp credit",
    "ze_kred1t",
    "sloncredit",
    "bingocash",
    "cash point",
    "cashberry",
    "slon credi",
    "domonet",
    "yoomoney",
    "creditpod-",
    "slon credit",
    "moneta-z",
    "bankacredi",
    "cash-kf",
    "advcash",
    "money.4.yo",
    "moneyexpert",
    "money-4-yo",
    "aviracredi",
    "money-4-you",
    "icredit",
    "creditbox",
    "money.4.you",
    "credit",
    "zekredi",
    "el caso",
    "creditup",
    "monetka",
    "dengivsim",
    "credit365",
    "moneyexper",
    "aviracredit",
    "kredit plu",
    "crediglbl",
    "bestcredit",
    "neocredit",
    "cashhelp",
    "telmone",
    "microcash",
    "novikredyt",
    "extramoney",
    "clycredit",
    "creditcafe",
    "ewacash",
    "kg-money",
    "creditik",
    "casharing",
    "case 24",
    "kreditstar",
    "forzacredit",
    "webmoney.ua",
    "forzacredi",
    "kredit plus",
    "ecase",
    "caseshop",
    "webmoney.u",
    "macincase",
    "verocash",
    "techno-cas",
    "novikredyty",
    "cash24",
    "onlycredit",
    "portmone",
    "mrmoney",
    "globalcredi",
    "credit-pro",
    "cashdesk",
    "stormoney",
    "credit2u",
    "opencredit",
    "smartmoney",
    "kvk-cash",
    "hotcredit",
    "techno-case",
    "cash365",
    "bestcredits",
    "bankacredit",
    "cash-ua",
    "sweet mone",
    "kredit1",
    "cashinua",
    "sweetmoney",
    "tviy cash",
    "moneyua",
    "catcredit",
    "money.spac",
    "topcredit",
    "sweet money",
    "pvks-kredit",
    "fotocaseua",
    "cashinsky",
    "easy cash",
    "yourmoney",
    "credbox",
    "kreditavans",
    "credit24",
    "casinoin",
    "globalcred",
    "caseller",
    "n1casino",
    "recredit",
    "happycredi",
    "uitracredit",
    "onecase",
    "seacredit",
    "smartcredi",
    "moneysmash",
    "pvks-kredi",
    "gdcashmere",
    "cool credi",
    "mlcrocredit",
    "turbocash",
    "allrightcas",
    "bystro.cas",
    "kreditavan",
    "blago cash",
    "ultracredi",
    "e-cash",
    "creditmax",
    "money_poin",
    "icases.ua",
    "dengi24",
    "hit_cash",
    "credit ok",
    "intercash",
    "monese",
    "elitcredit",
    "ify.credit",
    "dorcas",
    "moneyglad",
    "ifycredit",
    "uakredit",
    "bystro.cash",
    "kreditor",
    "dengidengi",
    "cashbe",
    "micredit",
    "casemaniac",
    "crazy case",
    "silver_cas",
    "sens credit",
    "mlcrocredi",
    "casey",
    "atom case",
    "cascata",
    "cashbox",
    "luckycash",
    "creditavir",
    "glad4money",
    "casekey",
    "kredit vsem",
    "multicast",
    "ultracashua",
    "hypno-casa",
    "avanscredit",
    "money_help",
    "casofficial",
    "incredo",
    "jeanscasual",
    "4-cases",
    "cool credit",
    "simplemone",
    "moneyup",
    "unicredit",
    "secretcase",
    "silvermone",
    "moneyjar",
    "nillkincase",
    "oncredit",
    "pancredit",
    "the-credit",
    "kredit_112",
    "flashcash",
    "creditbot",
    "creditorxx",
    "money_flas",
    "elitcases",
    "money_flash",
    "globalmone",
    "selectmoney",
    "prostomone",
    "kredenscafe",
    "moneyvalue",
    "sens credi",
    "money_star",
    "1case",
    "credit4u",
    "uitracredi",
    "mistermoney",
    "credit bot",
    "micrediton",
    "bit_money",
    "monet home",
    "tuskcasino",
    "casofficia",
    "prostomoney",
    "ultracredit",
    "reficredit",
    "ab.case",
    "money club",
    "moneysend",
    "creditnice1",
    "creditlite",
    "personcase",
    "bananacred",
    "likemoney",
    "creditavira",
    "pocketmone",
    "new_money",
]


food_delivery_companies = [
    "dominos",
    "sushi wok",
    "sushimaster",
    "sushi icons",
    "pizza 33",
    "sushimaste",
    "sushimaste",
    "glovo",
    "budusushi",
    "smilefood",
    "sushimaster",
    "sushi-poin",
    "sushi boss",
    "sushimaste",
    "sushimaste",
]


health_companies = [
    "e-health",
    "apteka911",
    "synevo",
    "helsi",
    "medcard24",
    "24/7 лікар",
    "med-servic",
    "med-service",
    "medcity.ua",
    "liki24.com",
    "apteka24.ua",
    "aptekanetu",
    "apteka d.s.",
    "podorozhnyk",
    "med-servlc",
    "apteka nc",
    "podorozhny",
    "med-servlce",
    "likar.info",
    "medcity",
    "likar",
    "apteka d.s",
    "leleka",
]

delivery_companies = [
    "novaposhta",
    "ukrposhta",
    "meestua",
    "global24",
]

competitors_companies = [
    "kyivstar",
    "vodafone ua",
    "lifecell",
    "kyivdigital",
]

competitors_provider_companies = [
    "ukrtelecom",
    "triolan",
    "datagroup",
    "langate",
    "fregat.com",
    "volia",
    "vega",
    "viasat",
]

shops_companies = [
    "rozetka",
    "prom.ua",
    "allo",
    "olx",
    "makeup",
    "citrus.ua",
    "stylus",
    "eldorado",
    "comfy",
    "foxtrot.ua",
    "epicentrk",
    "colins",
    "eva",
    "yakaboo",
    "exist.ua",
    "metro",
    "eva-mozayk",
    "mycredit.ua",
    "epic games",
    "bonjour",
    "citrus",
    "dzvlnok",
    "domino's",
    "fast box",
    "gift-servi",
    "kids-room",
    "kioto",
    "lemon.box",
    "link.dating",
    "mall",
    "medav",
    "ohlala",
    "photo-room",
    "pond",
    "rozetka, магазин",
    "shopster",
    "sofa_dream",
    "stay.cafe",
    "студия",
    "студия.меб",
    "ушастик",
    "холдинг",
    "цифровой",
    "шары",
    "shop_zakaz",
    "top-shop",
    "віці",
    "інтернет",
    "юа_магазин",
]

messengars_companies = [
    "viber",
    "whatsapp",
    "telegram",
    "facebook",
    "google",
    "instagram",
    "tiktok",
    "snapchat",
    "linkedin",
    "zoom",
    "discord",
    "twitter",
]

cyberpolice_companies = [
    "cyberpolice",
    "cyberpolic",
]

vodafone_support = [
    "111",
]

vodafone_spam = [
    "273",
    "275",
    "277",
]

vodafone_new_customer = [
    "222",
]

vodafone_services = [
    "30094",
    "7777",
    "30094",
    "5010",
    "1020",
    "2828",
    "30042",
    "vodafone u",
]

fraud_numbers = [
    "380800305555",
    "380442020202",
    "380444950405",
    "380445276363",
    "380445370222",
    "380442220333",
    "380442009010",
    "380442204404",
    "380442901988",
    "380567878104",
    "380666163133",
    "380677880884",
    "380957321212",
    "380958788181",
    "380443519998",
    "380995055757",
    "380800210800",
]

bank_numbers = [
    "380800500500",
    "380800500850",
    "380800307010",
    "380800502050",
    "380800504450",
    "380800502030",
    "380800507700",
    "380800504400",
    "380800309000",
    "380800307030",
    "380442907290",
    "380443630133",
    "380444908888",
    "380444900500",
    "380962907290",
    "729",
    "3700",
]

competitors_numbers = [
    "380674660466",
    "380997344444",
    "380505022250",
]

rescue_numbers = [
    "112",
    "102",
    "103",
    "dsns ukr",
]

verify_numbers = [
    "verify",
]

phone_companies = [
    "xiaomi",
    "apple",
    "samsung",
    "huawei",
]

In [225]:
bnum_categories_dict = {
    "bank_companies": bank_companies,
    "work_companies": work_companies,
    "grocery_companies": grocery_companies,
    "post_companies": post_companies,
    "taxy_companies": taxy_companies,
    "credits_companies": credits_companies,
    "food_delivery_companies": food_delivery_companies,
    "health_companies": health_companies,
    "delivery_companies": delivery_companies,
    "competitors_companies": competitors_companies,
    "competitors_provider_companies": competitors_provider_companies,
    "shops_companies": shops_companies,
    "messengars_companies": messengars_companies,
    "cyberpolice_companies": cyberpolice_companies,
    "vodafone": vodafone_support + vodafone_spam + vodafone_services + vodafone_new_customer,
    # "vodafone_support": vodafone_support,
    # "vodafone_spam": vodafone_spam,
    # "vodafone_services": vodafone_services,
    # "vodafone_new_customer": vodafone_new_customer,
    "fraud_numbers": fraud_numbers,
    "bank_numbers": bank_numbers,
    "competitors_numbers": competitors_numbers,
    "rescue_numbers": rescue_numbers,
    "verify_numbers": verify_numbers,
    "phone_companies": phone_companies,
}

df_train_bnum["bnum_category"] = "other"

for category, numbers in bnum_categories_dict.items():
    df_train_bnum["bnum_category"] = np.where(
        df_train_bnum["bnum"].isin(numbers), category, df_train_bnum["bnum_category"]
    )

In [226]:
others_bnums = (
    df_train_bnum[df_train_bnum.bnum_category == "other"]
    .bnum.value_counts()
    .sort_values(ascending=False)
    .index.to_list()
)

In [227]:
import re


def check_word(word):
    return bool(re.search(r"грош|mone|деньг|deng|cas|cred|кред|kred", word, re.IGNORECASE))


credit_phrases = []

for phrase in others_bnums:
    if check_word(phrase):
        credit_phrases.append(phrase)

credit_phrases

[]

In [228]:
churn_true = df_train_bnum[df_train_bnum["target"] == 1]
churn_false = df_train_bnum[df_train_bnum["target"] == 0]

In [229]:
churn_true.bnum_category.value_counts(normalize=True).sort_values(ascending=False).head(10)

bnum_category
other                    0.339511
cyberpolice_companies    0.150509
vodafone                 0.145784
bank_companies           0.098900
messengars_companies     0.085743
credits_companies        0.033238
taxy_companies           0.031650
rescue_numbers           0.021385
bank_numbers             0.021018
delivery_companies       0.016701
Name: proportion, dtype: float64

In [230]:
churn_false.bnum_category.value_counts(normalize=True).sort_values(ascending=False).head(10)

bnum_category
other                    0.395539
cyberpolice_companies    0.156486
bank_companies           0.109495
vodafone                 0.065129
messengars_companies     0.054749
taxy_companies           0.047852
delivery_companies       0.029451
credits_companies        0.026083
rescue_numbers           0.024446
health_companies         0.019776
Name: proportion, dtype: float64

In [6]:
churn_true.groupby("bnum").count().sort_values("target", ascending=False)

,call_cnt_out,call_cnt_in,call_dur_out,call_dur_in,cnt_sms_out,cnt_sms_in,abon_id,target
bnum,,,,,,,,
ukraine,2141,2141,2141,2141,2141,2141,2141,2141
cyberpolice,1927,1927,1927,1927,1927,1927,1927,1927
cyberpolic,1768,1768,1768,1768,1768,1768,1768,1768
1020,1404,1404,1404,1404,1404,1404,1404,1404
privatbank,1048,1048,1048,1048,1048,1048,1048,1048
...,...,...,...,...,...,...,...,...
8101,1,1,1,1,1,1,1,1
7979 taxi,1,1,1,1,1,1,1,1
7779,1,1,1,1,1,1,1,1


In [4]:
churn_false.groupby("bnum").count().sort_values("target", ascending=False)

,call_cnt_out,call_cnt_in,call_dur_out,call_dur_in,cnt_sms_out,cnt_sms_in,abon_id,target
bnum,,,,,,,,
cyberpolice,51513,51513,51513,51513,51513,51513,51513,51513
cyberpolic,49686,49686,49686,49686,49686,49686,49686,49686
ukraine,37578,37578,37578,37578,37578,37578,37578,37578
privatbank,29767,29767,29767,29767,29767,29767,29767,29767
taxi-838,20784,20784,20784,20784,20784,20784,20784,20784
...,...,...,...,...,...,...,...,...
380993885722,1,1,1,1,1,1,1,1
380674635604,1,1,1,1,1,1,1,1
380674635504,1,1,1,1,1,1,1,1
